In [ ]:
#강형민
#법정동코드 조회자료.csv 사용 / 동사무소real.csv 사용
KAKAO_API_KEY = "1469df95894d229baad7ac36bdd4175f"
KAKAO_ADDRESS_SEARCH_URL = "https://dapi.kakao.com//v2/local/search/keyword?query={}&page={}"

In [ ]:
#법정동코드 불러오기
import pandas as pd
bdong = pd.read_csv('/content/법정동코드 조회자료.csv',encoding='utf-8')
bdong

In [ ]:
#법정동코드 전처리
bdong = bdong.astype({'법정동코드':'int'})
bdongn = bdong[bdong['법정동코드']%1000000 == 0].index
bdong1 = bdong.drop(bdongn)
#print(bdong1)
#print(bdong1['법정동명'][0:])
bdong2 = bdong1['법정동명'][0:]
bdong3 = bdong2.iloc[[2]]
print(bdong3)

In [ ]:
#인증 정보 받아오기
header = {
  "Authorization": "KakaoAK {}".format(KAKAO_API_KEY)
}

header

In [ ]:
import requests
import json
i = 0
datas = pd.DataFrame()
gulist = bdong2

#위에서 받은 ~시 ~구 ~동 + 동사무소로 검색. 페이지 3까지 볼수있지만 검색어가 자세해서 1페이지만 해도 중복 많이 나올거 같았음
for i in range (len(bdong2.index)): 
    page=1
    response = requests.get(KAKAO_ADDRESS_SEARCH_URL.format("{} 동사무소".format(gulist.values[i]), page), headers=header) # 요청 URL 및 헤더 정보 넣어주기
    datas1 = response.json()
    documents = datas1['documents']
    dt1 = pd.DataFrame(documents)
    datas = pd.concat([datas, dt1])

    print("470번중",i,"번째")#테스트용


# response = requests.get(KAKAO_ADDRESS_SEARCH_URL.format("{} 동사무소".format(gulist.values[i]), page), headers=header) # 요청 URL 및 헤더 정보 넣어주기
# datas = response.json()
# datas
# for i in range (len(bdong2.index)):
#     page = 1
#     while True:
#         try:
#             response = requests.get(KAKAO_ADDRESS_SEARCH_URL.format("{} 동사무소".format(gulist.values[i]), page), headers=header) # 요청 URL 및 헤더 정보 넣어주기
#             datas1 = response.json()
#             datas.update(datas1)  
#             print(page,"페이지")
#             page += 1
#         except Exception as e:
#             print("오륜데용", e)
#             break


In [ ]:
#뽑은 dataframe 확인
datas

In [ ]:
#id 기준으로 중복 제거
datasd = datas.drop_duplicates(['id'])
datasd

In [ ]:
#일단 저장후 다시 전처리
datasd.to_csv("동사무소.csv", mode='a', header = False, encoding='utf-8-sig')
#엑셀에서 한글 깨져서 데이터 > 데이터 가져오기 > txt/csv에서 잘 만져서 다시 가져옴(어차피 다시 빼낼땐 또 깨질수도)

In [ ]:
#다시 불러왔음
ddata = pd.read_csv("동사무소real.csv", encoding='cp949') #utf-8 안되서 cp949로 불러옴
ddata

In [ ]:
#필요한 것만 뽑아내기
ddata = ddata[['Column9','Column2','Column12', 'Column13' ]]
ddata

In [ ]:
#Column명 변경
ddata.columns = ['건물명','주소','경도','위도']
ddata

In [ ]:
#경기도 스파이 제거

onlyseoul = ddata['주소'].str.contains("서울")
onlys = ddata[onlyseoul]
onlys

In [ ]:
#진짜 저장
onlys.to_csv("동사무소_전처리완료.csv", mode='a', header = False, encoding='utf-8-')